In [1]:
import math
import torch
from diffusion_edf.dist import IgSO3Dist, isotropic_gaussian_so3, isotropic_gaussian_so3_small_angle, isotropic_gaussian_so3_lie_deriv
from diffusion_edf import transforms
import plotly.graph_objs as go

In [2]:
dist = IgSO3Dist()

In [3]:
dtype = torch.float64
device = 'cpu'

eps = 1e-8
q = dist.sample(eps = eps, N=3, dtype=dtype, device=device)
# theta = 0.1
# # q = torch.tensor([[math.cos(theta/2), math.sin(theta/2), 0., 0.]], dtype=dtype, device=device)
# randomaxis = torch.randn(1,3, dtype=dtype, device=device)
# randomaxis = randomaxis / randomaxis.norm(dim=-1, keepdim=True) * math.sin(theta/2)
# q = torch.cat([torch.ones_like(randomaxis[...,:1]) * math.cos(theta/2), randomaxis], dim=-1)

prob = isotropic_gaussian_so3(q, eps)
deriv = isotropic_gaussian_so3_lie_deriv(q, eps)
print(f"Anaylitic Lie-derivative: {deriv}")

dt = 0.0000001

prob_rot = []
for i in range(3):
    qrot = transforms.quaternion_multiply(q,transforms.axis_angle_to_quaternion(dt * torch.eye(3, device=q.device, dtype=q.dtype)[i]))
    prob_rot.append(isotropic_gaussian_so3(qrot, eps))

prob_rot = torch.stack(prob_rot, dim=-1)
print(f"Numerical Lie-derivative: {(prob_rot - prob[...,None]) / dt}")
print(f"Allclose: {torch.allclose((prob_rot - prob[...,None]) / dt, deriv, atol=0, rtol=1e-2)}")

Anaylitic Lie-derivative: tensor([[ 2.1928e+15,  2.7360e+15, -5.9626e+15],
        [ 3.9635e+15,  1.8925e+15, -1.4181e+15],
        [ 5.8913e+13, -1.0603e+15, -3.1327e+15]], dtype=torch.float64)
Numerical Lie-derivative: tensor([[ 2.1913e+15,  2.7347e+15, -5.9619e+15],
        [ 3.9647e+15,  1.8921e+15, -1.4187e+15],
        [ 5.8361e+13, -1.0606e+15, -3.1312e+15]], dtype=torch.float64)
Allclose: True
